In [218]:
using MAGEMin_C
using DataFrames
using CSV
using drop

ArgumentError: ArgumentError: Package drop not found in current path.
- Run `import Pkg; Pkg.add("drop")` to install the drop package.

In [187]:
db = "ig"
sys_in  = "wt"

# pressure in kbar
P = 50.
# temperature 
T = LinRange(2200, 1000, 20)
buffer = "qfm"
B= -5.

database = Initialize_MAGEMin(db, verbose=false, buffer=buffer)

dfCompositions = CSV.read("thermoInput.csv", DataFrame)

# set oxide order based off of data frame, catching the meta data at beginning
Xoxides = names(dfCompositions[1,2:end])

cOutputs = []
for row in eachrow(dfCompositions)
    X = Array(row[2:end])
    tOutputs = []
    for t in T
        tout = single_point_minimization(P, t, database, X=X, Xoxides=Xoxides, sys_in=sys_in, B=B)
        push!(tOutputs, tout)
    end
    push!(cOutputs, (row, tOutputs))
end

In [ ]:
phs = [ph.ph for ph in [cOutputs[i][2][end] for i in eachindex(cOutputs)]]
uniquePhs = unique(mapreduce(permutedims, hcat, phs))
colnames = vcat(["coreSi", "mgsi", "P", "T"], uniquePhs)

dfSubSol = DataFrame([name => Float64[] for name = colnames])
for i in eachindex(cOutputs)
    coreSi = cOutputs[i][1][1]
    mgsi = cOutputs[i][1].MgO / cOutputs[i][1].SiO2 * ((28.084 + 15.999*2) / (24.305 + 15.9994))
    t = cOutputs[i][2][end].T_C
    p = cOutputs[i][2][end].P_kbar
    push!(dfSubSol, vcat([coreSi, mgsi, p, t], zeros(length(uniquePhs)) ))
    for (j, p) in enumerate(cOutputs[i][2][end].ph)
        if p != "qfm"
            dfSubSol[end, p] = cOutputs[i][2][end].ph_frac_wt[j]
        end
    end
end

CSV.write("subSol.csv", dfSubSol)

In [390]:
dfSubSol

Row,coreSi,mgsi,P,T,g,opx,cpx,coe,qfm,ol
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.665019,50.0,1000.0,0.0104488,0.722464,0.116698,0.150389,0.0,0.0
2,0.0634135,0.667516,50.0,1000.0,0.0104737,0.724215,0.117002,0.148309,0.0,0.0
3,0.126747,0.670031,50.0,1000.0,0.0104999,0.726025,0.117256,0.146219,0.0,0.0
4,0.19,0.672565,50.0,1000.0,0.0105253,0.727746,0.117612,0.144117,0.0,0.0
5,0.253172,0.675119,50.0,1000.0,0.0105518,0.729599,0.117843,0.142006,0.0,0.0
6,0.316265,0.677692,50.0,1000.0,0.0105774,0.731397,0.118141,0.139884,0.0,0.0
7,0.379278,0.680285,50.0,1000.0,0.0106039,0.733223,0.118422,0.137751,0.0,0.0
8,0.442212,0.682897,50.0,1000.0,0.0106303,0.73501,0.118751,0.135608,0.0,0.0
9,0.505066,0.68553,50.0,1000.0,0.0106569,0.736885,0.119003,0.133455,0.0,0.0


In [320]:
push!(dfSubSol, [1,1,1,1,1,1,1,1,1,1])

Row,coreSi,mgsi,T,P,g,opx,cpx,coe,qfm,ol
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1
